In [37]:
import cv2
import math
from win32api import GetSystemMetrics
import time
import numpy as np
import os

In [38]:
imagePath = 'Images'
# Image path
imageName = "sample3.PNG"
imagePath = os.path.join(imagePath,imageName)
#road detection works better with images where the scale is smaller per unit pixel 
#(screen distance = lesser real world distance)
#It also works better with a different style of image

mountingPoints = []
quadPoints = []

Steps to do test/implement ray masking
1. Plot out viewing quadrilateral
2. Plot out mounting point
3. Mask out for building contours
4. Find corners of building contours

Taking the nearest corner, find the slope of the line joining the mounting point and this nearest corner. Find the intersection point of this line and the farthest edge of the quadrilateral (What if farthest edge is out of bounds ??)

In [39]:
def getBorderContour_text(img):
#function runs only once
#using new bgr values for the new image in low red.

    img_c = img.copy()
    # Upper and lower color limit customized for snazzy maps
    low_red = (55, 55, 255)
    high_yellow = (242,251,256)

    low_gray = (241,241,241)
    high_gray = (244,243,241)

    # create masks
    red_mask = cv2.inRange(img, low_red, low_red )
    
    # combine masks
    kernel = np.ones((3,3), dtype=np.uint8)
    combined_mask = cv2.morphologyEx(red_mask, cv2.MORPH_DILATE,kernel)

    blank = np.zeros(img.shape, dtype='uint8')

    masked = cv2.bitwise_and(img,img,mask=combined_mask)

    # findcontours
    cnts=cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        # cv2.drawContours(img, [c], -1, (255,0,255), thickness=1)
        area = cv2.contourArea(c)
        if(area>200):
            for eps in np.linspace(0.001, 0.01, 10):
                # approximate the contour
                peri = cv2.arcLength(c, True)
                approx = cv2.approxPolyDP(c, eps * peri, True)
            
            # draw the approximated contour on the image  
            cv2.drawContours(blank, [approx], -1, (255,255,255), thickness=1)
            # cv2.drawContours(blank, [c], -1, 255, thickness=1)


    # cv2.imshow("image",img)
    # cv2.waitKey(0)

    return blank, masked

In [40]:
def click_MountingnScale_points(event, x, y, flags, params):

    # checking for left mouse clicks
    if event == cv2.EVENT_LBUTTONDOWN:
        mountingPoints.append((x,y))

        # displaying the coordinates on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, str(x) + ',' +str(y), (x,y), font,1, (255, 0, 0), 2)
        cv2.imshow('img', img)


    # checking for right mouse clicks    
    if event==cv2.EVENT_RBUTTONDOWN:
        quadPoints.append((x,y))

        # displaying the coordinates on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, str(x) + ',' +str(y),(x,y), font, 1,(0, 0, 255), 2)
        cv2.imshow('img', img)

In [41]:
def getCorners(buildingBorderMask, quadPts, blocksize=3):

    # ~~~~~~~~~~~~~ #
    # for obtaining corners for ray masking
    # ~~~~~~~~~~~~~ #

    # Mask may have 3 layers
    operatedImage = cv2.cvtColor(buildingBorderMask, cv2.COLOR_BGR2GRAY)

    # uncomment if you feel image is noisy (not needed)
    # operatedImage = cv2.fastNlMeansDenoisingColored(operatedImage,None,10,10,7,21) 

    # modify the data type -- setting to 32-bit floating point
    operatedImage = np.float32(operatedImage)

    # apply the cv2.cornerHarris method to detect the corners with appropriate values as input parameters
    # increase the second parameter ~ blocksize to get more of the corner shape out
    dest = cv2.cornerHarris(operatedImage, blocksize, 5, 0.07)  

    # Results are marked through the dilated corners
    dest = cv2.dilate(dest, None)

    # a mask of all corners of the building contours
    cornerMask = np.zeros(buildingBorderMask.shape, dtype='uint8')
    cornerMask[dest > 0.01 * dest.max()]=[255, 255, 255]

    # Make a gray scale mask of quadilateral area
    quadMask = np.zeros(buildingBorderMask.shape[:2], dtype='uint8')
    polyPts = np.array( quadPts ,dtype=np.int32)
    polyPts = polyPts.reshape((-1, 1, 2))
    quadMask = cv2.fillPoly(quadMask, pts=[polyPts],color=255)

    # a mask of all corners in quadilateral only
    cornerMask = cv2.bitwise_and(cornerMask,cornerMask,mask=quadMask)

    return cornerMask


In [42]:
def findClusterCenters(img, excludedCornerPoints = []):

    corner_centers = []

    onePixCornerMask = np.zeros(img.shape, dtype='uint8')
    # blank_copy = blank.copy()

    # create masks
    white_mask = cv2.inRange(img, (255,255,255), (255,255,255))

    # combine masks
    kernel = np.ones((3,3), dtype=np.uint8)
    combined_mask = cv2.morphologyEx(white_mask, cv2.MORPH_DILATE,kernel)


    cnts=cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for i in cnts:
        M = cv2.moments(i)
        if M['m00'] != 0:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
        # print(f"center - {cx},{cy}")
        if([cx,cy] not in excludedCornerPoints):
            onePixCornerMask[cy, cx] = [255, 255, 255]
            # cv2.circle(onePixCornerMask, (cx, cy), 1, (255, 255, 255), -1)
            corner_centers.append([cx,cy])

    onePixCornerMask = cv2.cvtColor(onePixCornerMask, cv2.COLOR_BGR2GRAY)

    return corner_centers, onePixCornerMask


In [43]:
def getCorners_w_exclude(img, cornerPoints=[]):
    
    operatedImage = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # modify the data type
    # setting to 32-bit floating point
    # operatedImage = cv2.fastNlMeansDenoisingColored(operatedImage,None,10,10,7,21) #uncomment if you feel image is noisy (not needed)
    # operatedImage = np.float32(operatedImage)

    # apply the cv2.cornerHarris method
    # to detect the corners with appropriate
    # values as input parameters
    #~~~~~~~~~~~~~#
    #for obtaining corners for ray masking
    #~~~~~~~~~~~~~#
    dest = cv2.cornerHarris(operatedImage, 2, 5, 0.07)

    # Results are marked through the dilated corners
    dest = cv2.dilate(dest, None)

    # for pts in poly_pts:
    #     dest[pts[0]][pts[1]] = 0
    # Reverting back to the original image,
    # with optimal threshold value
    size = img.shape

    cornerMask = np.zeros(size, dtype='uint8')
    cornerMask[dest > 0.01 * dest.max()]=[255, 255, 255]

    points,cornerMask = findClusterCenters(cornerMask,cornerPoints)

    return cornerMask, points

In [44]:
# read and display image 
img = cv2.imread(imagePath)
cv2.imshow("img",img)

# right click = view quadilateral (4+)
# left click = mounting point (1)
cv2.setMouseCallback('img', click_MountingnScale_points)
# points are to be selected in the order - 
# click 1 = nearest edge left point = quadPoints[0]
# click 2 = nearest edge right point = quadPoints[1]
# click 3 = further edge right point = quadPoints[2]
# click 4 = further edge left point = quadPoints[3]

cv2.waitKey(0)
cv2.destroyAllWindows()

In [45]:
# read image 
img = cv2.imread(imagePath)

# building contours on black image and building mask
bldg_brdrs, bldg_mask = getBorderContour_text(img)

# gather quadrilateral points
polyPts = np.array( quadPoints ,dtype=np.int32)
polyPts = polyPts.reshape((-1, 1, 2))

# make viewing quadrilateral mask
quadMask = np.zeros(img.shape[:2], dtype='uint8')
quadMask = cv2.fillPoly(quadMask, pts=[polyPts],color=255)

# get building borders inside viewing quadrilateral
selected_bldg_brdrs = cv2.bitwise_and(bldg_brdrs,bldg_brdrs, mask = quadMask)
selected_bldg_brdrs_gray = cv2.cvtColor(selected_bldg_brdrs, cv2.COLOR_BGR2GRAY) # Gray

# 
selected_edge_list = cv2.findContours(selected_bldg_brdrs_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
selected_edge_list = selected_edge_list[0] if len(selected_edge_list) == 2 else selected_edge_list[1]

# all_selected_corner = cv2.bitwise_and(bldg_brdrs, quadMask)
# all_selected_corner = getCorners(all_selected_corner)

# building corners inside viewing quadrilateral
original_corners = getCorners(bldg_brdrs, quadPoints)
original_corners_g = cv2.cvtColor(original_corners, cv2.COLOR_BGR2GRAY)
_, cornerPix = findClusterCenters(original_corners)

# Points displayed on img
# imgCornerPix = np.add(img, original_corners)

cv2.imshow("bldg_brdrs",cornerPix)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [46]:
# # use two masks to distinguish points on the side edges vs points on the nearest edge
# side_edge_mask = np.zeros(img.shape[:2], dtype='uint8')
# side_edge_mask = cv2.line(side_edge_mask, quadPoints[1], quadPoints[2], 255, 2) #plot right edge
# side_edge_mask = cv2.line(side_edge_mask, quadPoints[0], quadPoints[3], 255, 2)   #plot left edge

# nearest_edge_mask = np.zeros(img.shape[:2], dtype='uint8')
# nearest_edge_mask = cv2.line(side_edge_mask, quadPoints[0], quadPoints[1], 255, 2)   #plot nearest edge
# nearest_edge_corner = cv2.bitwise_and(all_selected_corner, nearest_edge_mask)

In [47]:
# initiate loop vars
quad_outline = np.zeros(bldg_brdrs.shape[:2], dtype='uint8')
one_edge_pic = np.zeros(bldg_brdrs.shape, dtype='uint8')

#Doing further calculation based on selected_edge_list[0], but in main code loop over all contours
for selected_edge_contour in selected_edge_list:

    #there is a chance that there are 2 corners caught in one contour. What to do here ? Choose the more inn

    # border of viewing quadrilateral
    cv2.polylines(quad_outline, [polyPts],True, 255, 1)

    # draws only the selected countor of the for loop
    cv2.drawContours(one_edge_pic, [selected_edge_contour], -1, (255,255,255), thickness=1)

    # nearest_edge_points = cv2.bitwise_and(nearest_edge_points, )

    # finding corners on the selected loop contour
    highlightedCorner = getCorners(one_edge_pic, quadPoints)
    #for all corners in the list (real and generated)
    #real corners = actual building corners
    #generated corners = building corners generated by viewing quadrilateral cut offs
    cornerPoint, highlightedCorner = findClusterCenters(highlightedCorner)

    highlightedCorner = cv2.bitwise_and(original_corners_g,highlightedCorner)

    # intersection of viewing quadrilateral and selected contour
    sample = cv2.bitwise_and(one_edge_pic, one_edge_pic,mask = quad_outline)
    cornerCoord, sample = findClusterCenters(sample)

    #code in lines to check whether or not there are 2 points detected
    #sometimes there can be only 1 or even no points

    #initially set these 2 points to invalid coordinates. Fill each of these points depending on the number intersection points returned from 
    #image manipulations
    intersection_pt_1 = [-1,-1]
    intersection_pt_2 = [-1,-1]

    if(len(cornerCoord)>0):
        if(len(cornerCoord)>=1):
            intersection_pt_1 = [cornerCoord[0][0],cornerCoord[0][1]]
        if(len(cornerCoord)>=2):
            intersection_pt_2 = [cornerCoord[1][0],cornerCoord[1][1]]

    img_final = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    #order of points matters. instead of going direct for the quadrilateral fill, maybe use 2 triangles instead ?
    mask_1_pts = np.array( [intersection_pt_1, cornerPoint[0], intersection_pt_1_1, slope_intersection_pt] ,dtype=np.int32)
    mask_1_pts = mask_1_pts.reshape((-1, 1, 2))

    mask_1 = np.zeros(img_final.shape[:2], dtype='uint8')
    mask_1 = cv2.fillPoly(mask_1, pts=[mask_1_pts],color = 255)
    mask_1_inverted = cv2.bitwise_and(mask_1, quadMask)
    mask_1 = cv2.bitwise_not(mask_1_inverted)

cv2.imshow("selected corner",highlightedCorner)
cv2.imshow("selected contour",one_edge_pic)
cv2.imshow("selected intersections",sample)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [48]:
#for each point, need to decide which farthest edge point constitutes the last point
#eqs of side edges of quadrilateral
slope_left_edge = (quadPoints[3][1] - quadPoints[0][1])/(quadPoints[3][0] - quadPoints[0][0])
slope_right_edge = (quadPoints[2][1] - quadPoints[1][1])/(quadPoints[2][0] - quadPoints[1][0])
c_left_edge = quadPoints[3][1] - slope_left_edge*quadPoints[3][0]
c_right_edge = quadPoints[1][1] - slope_left_edge*quadPoints[1][0]

intersection_pt_1_1 = [-1,-1] #assign the point to the right most point of viewing quad initially, change if the condition below is satisfied
intersection_pt_2_2 = [-1,-1] #assign the point to the right most point of viewing quad initially, change if the condition below is satisfied

# what if one of the points lies on nearest edge/farthest edge. if both the points lie on the nearest/farthest edge
# ideally, for a given building, if one of the building endings lies on the left edge, the corresponding 4th ed 

if(intersection_pt_1[0]!=-1):   #if intersection_pt_1 has valid info inside it (i.e. if it was found)
    print(intersection_pt_1[1])
    print(slope_left_edge*intersection_pt_1[0] + c_left_edge)
    if(intersection_pt_1[1] == (slope_left_edge*intersection_pt_1[0] + c_left_edge)):   #check if intersection pt 1 lies on left edge
        intersection_pt_1_1 = quadPoints[3]
    else:
        intersection_pt_1_1 = quadPoints[2] #set to right edge point

if(intersection_pt_2[0]!=-1):   #if intersection_pt_2 has valid info inside it (i.e. if it was found)
    print(intersection_pt_2[1])
    print(slope_left_edge*intersection_pt_2[0] + c_left_edge)
    if(intersection_pt_2[1] == (slope_left_edge*intersection_pt_2[0] + c_left_edge)):   #check if intersection pt 2 lies on left edge
        intersection_pt_2_2 = quadPoints[3]
    else:
        intersection_pt_2_2 = quadPoints[2] #set to right edge point

357
16305.0
354
12517.5


In [50]:
mountingPoint = mountingPoints[0]

slope_farthest_edge = (quadPoints[3][1]-quadPoints[2][1])/(quadPoints[3][0]-quadPoints[2][0]) #slope of farthest edge line

c2 = quadPoints[3][1] - slope_farthest_edge*quadPoints[3][0] #constant of farthest edge line

slope = (mountingPoint[1] - cornerPoint[0][1])/(mountingPoint[0] - cornerPoint[0][0])
#find the intersection point b/w the farthest edge line and the line joining cornerPoint and mountingPoint
#points are selected such that quadPoints[3] and quadPoints[4] represent the points for the farthest edge

#finding intersection point of both lines associated w/ slope
#x_intercept = (c2-c1)/(m1-m2)
#y_intercept = m1*x_intercept + c1

c1 = mountingPoint[1] - slope*mountingPoint[0] #constant of line joining mounting point and corner = y1 - mx1

#have obtained one point for the blacked out region
x_intercept = (c2-c1)/(slope - slope_farthest_edge)
y_intercept = slope*x_intercept + c1

slope_intersection_pt = [x_intercept, y_intercept]

print(intersection_pt_1)
print(intersection_pt_1_1)
print(intersection_pt_2)
print(intersection_pt_2_2)
print(slope_intersection_pt)
print(quadPoints)

[487, 357]
(493, 357)
[412, 354]
(493, 357)
[488.18912462811346, 356.85059393255005]
[(165, 44), (486, 47), (493, 357), (171, 347)]


In [52]:
img = cv2.imread(imagePath)
img_final = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#order of points matters. instead of going direct for the quadrilateral fill, maybe use 2 triangles instead ?

mask_1_pts = np.array( [intersection_pt_1, cornerPoint[0], intersection_pt_1_1, slope_intersection_pt] ,dtype=np.int32)
mask_1_pts = mask_1_pts.reshape((-1, 1, 2))

mask_1 = np.zeros(img_final.shape[:2], dtype='uint8')
mask_1 = cv2.fillPoly(mask_1, pts=[mask_1_pts],color = 255)
mask_1_inverted = cv2.bitwise_and(mask_1, quadMask)
mask_1 = cv2.bitwise_not(mask_1_inverted)
#~~~~~~~~~#
#NOT IMPLEMENTED YET
#order of points matters. instead of going direct for the quadrilateral fill, maybe use 2 triangles instead ?
mask_2_pts = np.array( [intersection_pt_2, cornerPoint[0], intersection_pt_2_2,slope_intersection_pt] ,dtype=np.int32)
mask_2_pts = mask_1_pts.reshape((-1, 1, 2))

mask_2 = np.zeros(img_final.shape[:2], dtype='uint8')
mask_2 = cv2.fillPoly(mask_2, pts=[mask_2_pts],color = 255)
mask_2_inverted = cv2.bitwise_and(mask_2, quadMask)
mask_2 = cv2.bitwise_not(mask_2_inverted)
#~~~~~~~~#

combined_mask = np.add(mask_1_inverted, mask_2_inverted)
combined_mask = cv2.bitwise_not(combined_mask)

img_final = cv2.bitwise_and(img_final, mask_1)
img_final = cv2.bitwise_or(img_final, quad_outline)

mask_diff = np.subtract(mask_1, mask_2)

cv2.imshow("img final",img_final)
cv2.imshow("subtract mask 1",mask_1)
cv2.waitKey(0)
cv2.destroyAllWindows()